In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForPreTraining,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig,get_peft_model
from trl import SFTTrainer

/home/yujie/miniconda3/envs/llama2/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
!huggingface-cli login --token YOUR_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/yujie/.cache/huggingface/token
Login successful


In [8]:
# Model from Hugging Face hub
base_model = "meta-llama/Llama-2-7b-chat-hf"
base_model = "/mnt/c/independent_study/new_model"
# Fine-tuned model
new_model = "/mnt/c/independent_study/final_model"

In [4]:
# import pandas as pd
# data = pd.read_json("/mnt/c/independent_study/data/text.json")
# import json
# with open("/mnt/c/independent_study/data/new_text.json", 'w') as f:
#     for i in range(len(data)):
#         json.dump({'text': data['text'][i]}, f)

In [5]:
train = load_dataset('json', data_files="/mnt/c/independent_study/data/train.json", split="train")
val = load_dataset('json', data_files="/mnt/c/independent_study/data/val.json", split="train")
dataset = load_dataset('json', data_files="/mnt/c/independent_study/data/new_text.json", split="train")

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
    # llm_int8_enable_fp32_cpu_offload=False
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
for i in model.named_parameters():
    print(f"{i[0]} -> {i[1].device}")

model.embed_tokens.weight -> cuda:0
model.layers.0.self_attn.q_proj.lora_A.default.weight -> cuda:0
model.layers.0.self_attn.q_proj.lora_B.default.weight -> cuda:0
model.layers.0.self_attn.q_proj.base_layer.weight -> cuda:0
model.layers.0.self_attn.k_proj.weight -> cuda:0
model.layers.0.self_attn.v_proj.lora_A.default.weight -> cuda:0
model.layers.0.self_attn.v_proj.lora_B.default.weight -> cuda:0
model.layers.0.self_attn.v_proj.base_layer.weight -> cuda:0
model.layers.0.self_attn.o_proj.weight -> cuda:0
model.layers.0.mlp.gate_proj.weight -> cuda:0
model.layers.0.mlp.up_proj.weight -> cuda:0
model.layers.0.mlp.down_proj.weight -> cuda:0
model.layers.0.input_layernorm.weight -> cuda:0
model.layers.0.post_attention_layernorm.weight -> cuda:0
model.layers.1.self_attn.q_proj.lora_A.default.weight -> cuda:0
model.layers.1.self_attn.q_proj.lora_B.default.weight -> cuda:0
model.layers.1.self_attn.q_proj.base_layer.weight -> cuda:0
model.layers.1.self_attn.k_proj.weight -> cuda:0
model.layers

In [ ]:
# l = []
# for i in model.named_parameters():
#     if str(i[1].device) == 'cuda:0':
#         l.append(True)
#     else:
#         l.append(False)
#         print(f"{i[0]} -> {i[1].device}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

In [11]:
prompt = "what is support vector in SVM?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(f'''{result[0]['generated_text']}''')

In [12]:
prompt = "What is a true mean in statistics?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [13]:
prompt = "What is AUC curve in machine learning?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=3,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

In [ ]:
trainer = SFTTrainer(
    # peft_config=peft_params,
    model=model,
    train_dataset=dataset,
    eval_dataset=val,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [14]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


('/mnt/c/independent_study/final_model/tokenizer_config.json',
 '/mnt/c/independent_study/final_model/special_tokens_map.json',
 '/mnt/c/independent_study/final_model/tokenizer.json')

In [15]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))